In [43]:
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from src.scripts.preprocess import normalize_text, remove_stopwords

In [44]:
df = pd.read_csv(r'C:\Users\ritah\Documents\RepositoriosGithub\BACK-DOMROCK\src\data\chat_data.csv')

In [25]:
print(df.columns)

Index(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'overall_rating', 'recommend_to_a_friend',
       'review_text', 'reviewer_birth_year', 'reviewer_gender',
       'reviewer_state'],
      dtype='object')


In [26]:
# define as colunas do csv que serão usadas
useful_data = ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'overall_rating', 'review_text', 'reviewer_state']

In [45]:
# carrega o csv
def load_data(file_path):
    # lê o arquivo csv
    df = pd.read_csv(file_path)
    
    # exclui as colunas que não serão usadas
    df_reduced = df.drop(columns=[col for col in df.columns if col not in useful_data])

    # limpa e normaliza o texto
    for column in useful_data:
        df_reduced[column] = df_reduced[column].apply(lambda x: normalize_text(str(x)))
        df_reduced[column] = df_reduced[column].apply(lambda x: remove_stopwords(str(x)))

    return df_reduced

In [46]:
# Chama a função para carregar os dados e obter o DataFrame reduzido
df_reduced = load_data(r'C:\Users\ritah\Documents\RepositoriosGithub\BACK-DOMROCK\src\data\chat_data.csv')

In [47]:
# define o nome e o caminho do arquivo de saída
result_file_name = 'data_processed.csv'  # nome do arquivo de saída
new_file_path = os.path.join('data', result_file_name)  # caminho para salvar o arquivo de saída na pasta 

In [48]:
# Define o diretório onde o arquivo será salvo
data_dir = os.path.join('src', 'data')  # Caminho para a pasta 'src/data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)  # Cria a pasta 'src/data' se não existir

In [49]:
# Define o nome e o caminho do arquivo de saída
result_file_name = 'data_processed.csv'  # nome do arquivo de saída
new_file_path = os.path.join(data_dir, result_file_name)  # Caminho correto para salvar o arquivo em 'src/data'

In [50]:
# salva os dados em csv
df_reduced.to_csv(new_file_path, index=False)

In [51]:
# Retorna o caminho do arquivo gerado
print(f'Arquivo salvo em: {new_file_path}')

Arquivo salvo em: src\data\data_processed.csv


In [52]:
def load_and_chunk(file_path):
    # gera os documentos com o csv processado
    loader = CSVLoader(file_path=file_path, encoding='utf-8', csv_args={
        'delimiter': ',', 'quotechar': '"', 'fieldnames': useful_data
    }) # classe do langchain para manipular os dados
    # delimiter define o que separa as colunas do csv (no caso, uma vírgula)
    # quotechar define o caractere que envolve strings, permitindo que uma string tenha vírgulas sem ser divididas em colunas
    # fieldnames são os nomes das colunas que serão carregadas

    docs = loader.load() # retorna os documentos gerados
 
    # chunkeniza e adiciona overlap nos dados
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    splits = text_splitter.split_documents(docs)

    return splits

In [55]:
print(df_reduced.head())

                                        product_name   product_brand  \
0  notebook asus vivobook max x541nago472t intel ...             nan   
1                   copo acrilico canudo 500ml rocie             nan   
2  panela pressao eletrica philips walita daily 5...  philips walita   
3                 betoneira columbus roma brinquedos     roma jensen   
4  smart tv led 43 lg 43uj6525 ultra hd 4k conver...              lg   

       site_category_lv1    site_category_lv2 overall_rating  \
0            informatica             notebook              4   
1  utilidades domesticas  copos tacas canecas              4   
2        eletroportateis      panela eletrica              4   
3             brinquedos   veiculos brinquedo              4   
4        tv home theater                   tv              5   

                                         review_text reviewer_state  
0  contente compra entrega rapida unico problema ...             rj  
1  apenas r199420eu consegui comprar lindo

In [56]:
print(df_reduced.columns)

Index(['product_name', 'product_brand', 'site_category_lv1',
       'site_category_lv2', 'overall_rating', 'review_text', 'reviewer_state'],
      dtype='object')
